# Creating 2018 dataset - with extra columns and variables not included in 1994
#### 10/2/2020

## Load in data

#### import libraries and packages

In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn import preprocessing
#from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings('ignore')

#### Read in the data - be careful running this cell - data is 1 GB

In [2]:
df = pd.read_csv('Census_data/2018-1-year-raw/psam_pusa.csv').sample(frac=1)
df.head()

,RT,SERIALNO,DIVISION,SPORDER,PUMA,REGION,ST,ADJINC,PWGTP,AGEP,...,PWGTP71,PWGTP72,PWGTP73,PWGTP74,PWGTP75,PWGTP76,PWGTP77,PWGTP78,PWGTP79,PWGTP80
1417367,P,2018HU0397426,1,3,704,1,25,1013097,88,48,...,158,111,109,83,32,96,24,105,86,154
1450215,P,2018HU1093783,1,2,704,1,25,1013097,78,47,...,140,24,24,140,86,77,116,81,116,129
792094,P,2018HU1026262,5,1,1101,3,12,1013097,51,50,...,81,15,45,14,15,52,47,51,59,17
1083856,P,2018HU1188143,3,4,3308,2,17,1013097,219,6,...,225,73,236,213,225,412,218,215,66,226
1564904,P,2018GQ0074645,4,1,1301,2,27,1013097,34,19,...,60,61,36,34,4,33,34,34,33,37


In [4]:
#keep only columns of interest
#includes columns from original 1994 data, plus some new variables
df_2018 = df[['DIVISION','PUMA','REGION','ST','RT','PWGTP',
              'AGEP','CIT','COW','SCHL','MAR','OCCP','RELP','RAC1P','SEX','INTP',
              'WKHP','FOD1P','FOD2P','POBP','PINCP','SCIENGP']]
df_2018.head()

,DIVISION,PUMA,REGION,ST,RT,PWGTP,AGEP,CIT,COW,SCHL,...,RELP,RAC1P,SEX,INTP,WKHP,FOD1P,FOD2P,POBP,PINCP,SCIENGP
1417367,1,704,1,25,P,88,48,2,4.0,17.0,...,13,1,1,0.0,40.0,NaN,NaN,72,77000.0,NaN
1450215,1,704,1,25,P,78,47,1,1.0,22.0,...,1,1,2,0.0,40.0,5200.0,NaN,36,75000.0,1.0
792094,5,1101,3,12,P,51,50,1,1.0,21.0,...,0,1,1,0.0,50.0,6201.0,NaN,26,80000.0,2.0
1083856,3,3308,2,17,P,219,6,1,NaN,3.0,...,2,1,1,NaN,NaN,NaN,NaN,17,NaN,NaN
1564904,4,1301,2,27,P,34,19,1,1.0,18.0,...,17,1,2,0.0,8.0,NaN,NaN,27,4800.0,NaN


In [5]:
#filter data to only include people over 16, with income over 100, and who work more than 0 hours per week
df_2018 = df_2018[(df_2018['AGEP']> 16) & (df_2018['PINCP']>100) & (df_2018['WKHP']>0)]
df_2018.head()

,DIVISION,PUMA,REGION,ST,RT,PWGTP,AGEP,CIT,COW,SCHL,...,RELP,RAC1P,SEX,INTP,WKHP,FOD1P,FOD2P,POBP,PINCP,SCIENGP
1417367,1,704,1,25,P,88,48,2,4.0,17.0,...,13,1,1,0.0,40.0,NaN,NaN,72,77000.0,NaN
1450215,1,704,1,25,P,78,47,1,1.0,22.0,...,1,1,2,0.0,40.0,5200.0,NaN,36,75000.0,1.0
792094,5,1101,3,12,P,51,50,1,1.0,21.0,...,0,1,1,0.0,50.0,6201.0,NaN,26,80000.0,2.0
1564904,4,1301,2,27,P,34,19,1,1.0,18.0,...,17,1,2,0.0,8.0,NaN,NaN,27,4800.0,NaN
1557791,3,1100,2,26,P,107,24,1,1.0,21.0,...,0,1,2,0.0,40.0,5098.0,NaN,26,10000.0,1.0


In [6]:
df_2018.describe()

,DIVISION,PUMA,REGION,ST,PWGTP,AGEP,CIT,COW,SCHL,MAR,...,RELP,RAC1P,SEX,INTP,WKHP,FOD1P,FOD2P,POBP,PINCP,SCIENGP
count,847151.000000,847151.000000,847151.000000,847151.000000,847151.000000,847151.000000,847151.000000,847151.000000,847151.000000,847151.000000,...,847151.000000,847151.000000,847151.000000,847151.000000,847151.000000,307854.000000,34822.000000,847151.000000,8.471510e+05,307854.000000
mean,5.633533,3086.636708,2.916625,13.962962,104.086251,43.407701,1.585413,2.089781,18.638762,2.541756,...,2.262280,1.992258,1.479649,1833.709291,38.291273,4385.922856,4360.970163,63.560860,5.772235e+04,1.613797
std,2.563186,2993.935494,0.945998,7.819940,84.002080,15.262217,1.290035,1.844993,3.352682,1.802106,...,4.380358,2.201901,0.499586,17068.374505,13.065144,1718.909684,1708.574779,100.235554,7.454377e+04,0.486879
min,1.000000,100.000000,1.000000,1.000000,1.000000,17.000000,1.000000,1.000000,1.000000,1.000000,...,0.000000,1.000000,1.000000,-2400.000000,1.000000,1100.000000,1100.000000,1.000000,1.040000e+02,1.000000
25%,3.000000,803.000000,2.000000,6.000000,55.000000,30.000000,1.000000,1.000000,16.000000,1.000000,...,0.000000,1.000000,1.000000,0.000000,35.000000,2408.000000,2414.000000,12.000000,2.000000e+04,1.000000
50%,5.000000,2002.000000,3.000000,12.000000,81.000000,43.000000,1.000000,1.000000,19.000000,1.000000,...,1.000000,1.000000,1.000000,0.000000,40.000000,5200.000000,5098.000000,23.000000,3.970000e+04,2.000000
75%,8.000000,3765.000000,4.000000,21.000000,125.000000,56.000000,1.000000,3.000000,21.000000,5.000000,...,2.000000,1.000000,2.000000,0.000000,43.000000,6107.000000,6103.000000,41.000000,7.000000e+04,2.000000
max,9.000000,12704.000000,4.000000,28.000000,1990.000000,96.000000,5.000000,8.000000,24.000000,5.000000,...,17.000000,9.000000,2.000000,354000.000000,99.000000,6403.000000,6403.000000,554.000000,1.353400e+06,2.000000


In [7]:
# create csv

df_2018.to_csv('2018_new-var_mapped.csv',index=False)